In [3]:
import sys
import os

# Get the current directory of your notebook (or script)
# Assumes you are running the notebook from within the OpenAlpha_Evolve directory
notebook_dir = os.getcwd()

# Construct the path to the count-graph-homs directory
# It's a sibling directory to where you are running the notebook/script
count_graph_homs_path = os.path.join(notebook_dir, 'count-graph-homs')

# Add this path to the Python search path
if count_graph_homs_path not in sys.path:
    sys.path.append(count_graph_homs_path)

print(f"Added {count_graph_homs_path} to sys.path")


Added /Users/thomaskidane/Documents/Projects/openevovle/OpenAlpha_Evolve/count-graph-homs/count-graph-homs to sys.path


In [4]:
from standard_hom_count import GraphHomomorphismCounter


In [5]:
from sage.all import *
from standard_hom_count import GraphHomomorphismCounter # Assuming standard_hom_count.py is in the same directory or your Python path

# Define the fixed bipartite graph H (e.g., Complete Bipartite K_{2,3})
H = graphs.CompleteBipartiteGraph(2, 3)
print(f"Fixed bipartite graph H: {H.graph6_string()}") # Print graph representation


# Initialize a random graph G (e.g., a random graph with 5 vertices and edge probability 0.4)
num_vertices_G = 5
edge_probability_G = 0.4
G = graphs.RandomGNP(num_vertices_G, edge_probability_G)
print(f"Initial random graph G: {G.graph6_string()}") # Print graph representation

def calculate_homomorphism_density(H, G):
    """Calculates the homomorphism density from graph H to graph G."""
    if len(G.vertices()) == 0:
        return 0.0 # Density is 0 if the target graph has no vertices

    counter = GraphHomomorphismCounter(H, G)
    num_homomorphisms = counter.count_homomorphisms()

    # The normalization factor is |V(G)| raised to the power of |V(H)|
    normalization_factor = len(G.vertices())**len(H.vertices())
    

    if normalization_factor == 0:
        return float('inf') # Avoid division by zero if H has no vertices (shouldn't happen with K_{2,3})

    density = num_homomorphisms / normalization_factor
    return density

import random

def apply_random_edge_action(G):
    """Applies a random edge addition or removal action to graph G."""
    vertices = list(G.vertices())
    if len(vertices) < 2:
        # Cannot add/remove edge with less than 2 vertices
        return G.copy() # Return a copy to avoid modifying in place if action is impossible

    action_type = random.choice(['add', 'remove'])
    G_new = G.copy() # Work on a copy

    if action_type == 'add':
        # Try adding a random edge between two distinct vertices
        u, v = random.sample(vertices, 2)
        if not G_new.has_edge(u, v):
            G_new.add_edge(u, v)
            # print(f"Action: Added edge ({u}, {v})")
        # else:
            # print(f"Action: Tried to add edge ({u}, {v}), but it already exists.")
    elif action_type == 'remove':
        # Try removing a random edge
        edges = list(G_new.edges())
        if edges:
            u, v, label = random.choice(edges)
            G_new.delete_edge(u, v)
            # print(f"Action: Removed edge ({u}, {v})")
        # else:
            # print(f"Action: Tried to remove an edge, but the graph has none.")

    return G_new

# Calculate initial density and reward
initial_density = calculate_homomorphism_density(H, G)
initial_reward = -initial_density
print(f"Initial homomorphism density (H->G): {initial_density}")
print(f"Initial reward: {initial_reward}")

# --- Simulation Loop ---
current_G = G.copy()
num_simulation_steps = 10
best_G = current_G.copy()
best_reward = -calculate_homomorphism_density(H, current_G)

print("\nStarting simulation...")
for step in range(num_simulation_steps):
    # Apply a random action
    next_G = apply_random_edge_action(current_G)

    # Calculate reward for the new graph
    density = calculate_homomorphism_density(H, next_G)
    reward = -density

    print(f"\nStep {step + 1}:")
    # print(f"  Graph G after action: {next_G.graph6_string()}") # Optional: print graph string
    print(f"  Homomorphism density (H->G): {density}")
    print(f"  Reward: {reward}")

    # Update best graph if current reward is better
    if reward > best_reward:
        best_reward = reward
        best_G = next_G.copy()
        print("  New best graph found!")

    # Update current graph for the next step
    current_G = next_G.copy()

print("\nSimulation finished.")
def check_sidorenko_violation(H, G):
    """Checks if the Sidorenko conjecture is violated for H and G."""
    hom_density = calculate_homomorphism_density(H, G)
    edge_density_G = G.density()
    num_vertices_H = len(H.vertices())

    # Sidorenko's conjecture states hom_density(H, G) >= density(G)^|V(H)|
    # Violation occurs if hom_density(H, G) < density(G)^|V(H)|
    return hom_density < (edge_density_G ** num_vertices_H)
# Check for Sidorenko conjecture violation and display the graph if not violated
if check_sidorenko_violation(H, best_G):
    print("")
    print("Sidorenko conjecture *may* be violated for the best graph found.")
else:
    print("Sidorenko conjecture does *not* appear to be violated for the best graph found.")
    # Display the best graph
    show(best_G.plot())
# You can plot the best graph found:
# show(best_G.plot())

Fixed bipartite graph H: D]o
Initial random graph G: Djg
Initial homomorphism density (H->G): 0.0384
Initial reward: -0.0384

Starting simulation...

Step 1:
  Homomorphism density (H->G): 0.02816
  Reward: -0.02816
  New best graph found!

Step 2:
  Homomorphism density (H->G): 0.02816
  Reward: -0.02816

Step 3:
  Homomorphism density (H->G): 0.01472
  Reward: -0.01472
  New best graph found!

Step 4:
  Homomorphism density (H->G): 0.00448
  Reward: -0.00448
  New best graph found!

Step 5:
  Homomorphism density (H->G): 0.00384
  Reward: -0.00384
  New best graph found!

Step 6:
  Homomorphism density (H->G): 0.00064
  Reward: -0.00064
  New best graph found!

Step 7:
  Homomorphism density (H->G): 0.0
  Reward: -0.0
  New best graph found!

Step 8:
  Homomorphism density (H->G): 0.00064
  Reward: -0.00064

Step 9:
  Homomorphism density (H->G): 0.00384
  Reward: -0.00384

Step 10:
  Homomorphism density (H->G): 0.00064
  Reward: -0.00064

Simulation finished.
Sidorenko conjecture d

In [12]:
best_G

RandomGNP(5,0.4): Graph on 5 vertices

In [11]:
H

Complete bipartite graph of order 2+3: Graph on 5 vertices

In [1]:
from sage.all import *
from standard_hom_count import GraphHomomorphismCounter # Assuming standard_hom_count.py is in the same directory or your Python path
from decimal import Decimal, getcontext

# Set the precision for Decimal calculations
getcontext().prec = 50 # You can adjust the precision as needed

# Define the fixed bipartite graph H (e.g., Complete Bipartite K_{2,3})
H = graphs.CompleteBipartiteGraph(2, 3)
print(f"Fixed bipartite graph H: {H.graph6_string()}") # Print graph representation


# Initialize a random graph G (e.g., a random graph with 5 vertices and edge probability 0.4)
num_vertices_G = 5
edge_probability_G = Decimal('0.4') # Use Decimal for probability
G = graphs.RandomGNP(num_vertices_G, float(edge_probability_G)) # RandomGNP likely expects a float, convert back for graph generation
print(f"Initial random graph G: {G.graph6_string()}") # Print graph representation

def calculate_homomorphism_density(H, G):
    """Calculates the homomorphism density from graph H to graph G using Decimal for precision."""
    if len(G.vertices()) == 0:
        return Decimal('0.0') # Return Decimal zero

    counter = GraphHomomorphismCounter(H, G)
    num_homomorphisms = Decimal(counter.count_homomorphisms()) # Convert count to Decimal

    # The normalization factor is |V(G)| raised to the power of |V(H)|
    # Convert to Decimal before exponentiation
    normalization_factor = Decimal(len(G.vertices()))**len(H.vertices())


    if normalization_factor == Decimal('0.0'):
        return Decimal('inf') # Avoid division by zero if H has no vertices (shouldn't happen with K_{2,3})

    density = num_homomorphisms / normalization_factor
    return density

def check_sidorenko_violation(H, G):
    """Checks if the Sidorenko conjecture is violated for H and G using Decimal for precision."""
    hom_density = calculate_homomorphism_density(H, G)
    edge_density_G = Decimal(G.density()) # Convert edge density to Decimal
    num_vertices_H = len(H.vertices())

    # Sidorenko's conjecture states hom_density(H, G) >= density(G)^|V(H)|
    # Violation occurs if hom_density(H, G) < density(G)^|V(H)|
    return hom_density < (edge_density_G ** num_vertices_H)

import random

def apply_random_edge_action(G):
    """Applies a random edge addition or removal action to graph G."""
    vertices = list(G.vertices())
    if len(vertices) < 2:
        # Cannot add/remove edge with less than 2 vertices
        return G.copy() # Return a copy to avoid modifying in place if action is impossible

    action_type = random.choice(['add', 'remove'])
    G_new = G.copy() # Work on a copy

    if action_type == 'add':
        # Try adding a random edge between two distinct vertices
        u, v = random.sample(vertices, 2)
        if not G_new.has_edge(u, v):
            G_new.add_edge(u, v)
            # print(f"Action: Added edge ({u}, {v})")
        # else:
            # print(f"Action: Tried to add edge ({u}, {v}), but it already exists.")
    elif action_type == 'remove':
        # Try removing a random edge
        edges = list(G_new.edges())
        if edges:
            u, v, label = random.choice(edges)
            G_new.delete_edge(u, v)
            # print(f"Action: Removed edge ({u}, {v})")
        # else:
            # print(f"Action: Tried to remove an edge, but the graph has none.")

    return G_new

# Calculate initial density and reward
initial_density = calculate_homomorphism_density(H, G)
initial_reward = -initial_density
print(f"Initial homomorphism density (H->G): {initial_density}")
print(f"Initial reward: {initial_reward}")

# --- Simulation Loop ---
current_G = G.copy()
num_simulation_steps = 10
best_G = current_G.copy()
best_reward = -calculate_homomorphism_density(H, current_G)

print("\nStarting simulation...")
for step in range(num_simulation_steps):
    # Apply a random action
    next_G = apply_random_edge_action(current_G)

    # Calculate reward for the new graph
    density = calculate_homomorphism_density(H, next_G)
    reward = -density

    print(f"\nStep {step + 1}:")
    # print(f"  Graph G after action: {next_G.graph6_string()}") # Optional: print graph string
    print(f"  Homomorphism density (H->G): {density}")
    print(f"  Reward: {reward}")

    # Update best graph if current reward is better
    if reward > best_reward:
        best_reward = reward
        best_G = next_G.copy()
        print("  New best graph found!")

    # Update current graph for the next step
    current_G = next_G.copy()

print("\nSimulation finished.")
print(f"Best graph found (reward: {best_reward}): {best_G.graph6_string()}")

# Check for Sidorenko conjecture violation and display the graph if not violated
if check_sidorenko_violation(H, best_G):
    print("Sidorenko conjecture *may* be violated for the best graph found.")
else:
    print("Sidorenko conjecture does *not* appear to be violated for the best graph found.")
    # Display the best graph
    show(best_G.plot())

Fixed bipartite graph H: D]o
Initial random graph G: DO?
Initial homomorphism density (H->G): 0.00064
Initial reward: -0.00064

Starting simulation...

Step 1:
  Homomorphism density (H->G): 0.00128
  Reward: -0.00128

Step 2:
  Homomorphism density (H->G): 0.00064
  Reward: -0.00064

Step 3:
  Homomorphism density (H->G): 0
  Reward: 0
  New best graph found!

Step 4:
  Homomorphism density (H->G): 0
  Reward: 0

Step 5:
  Homomorphism density (H->G): 0.00064
  Reward: -0.00064

Step 6:
  Homomorphism density (H->G): 0
  Reward: 0

Step 7:
  Homomorphism density (H->G): 0
  Reward: 0

Step 8:
  Homomorphism density (H->G): 0.00064
  Reward: -0.00064

Step 9:
  Homomorphism density (H->G): 0
  Reward: 0

Step 10:
  Homomorphism density (H->G): 0
  Reward: 0

Simulation finished.
Best graph found (reward: 0): D??


TypeError: conversion from sage.rings.rational.Rational to Decimal is not supported